# Neural networks code and tests notebook

# This notebook takes the flux, photon index and fratio data and try to predict classes from this data on a :

# - LSTM network
# - ConvLSTM network
# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []

nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate

data_matrix= np.zeros((b,c,a))

#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][j]=dataframe['Flux']
    data_matrix[1][j]=dataframe['Photon Index']
    
for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][v]=dataframe['Flux']
    data_matrix[1][v]=dataframe['Photon Index']
    
#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))



In [6]:
#Reshaping data matrix
data_matrix=data_matrix.reshape(c,nbfeatures,binning)

In [7]:
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(500):
    
    model = Sequential()
    model.add(LSTM(9,input_shape=(nbfeatures,binning),activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])
    
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6434 - binary_accuracy: 0.5700
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5774 - binary_accuracy: 0.5568
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5816 - binary_accuracy: 0.5844
Epoch 4/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5436 - binary_accuracy: 0.5568
Epoch 5/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5125 - binary_accuracy: 0.5580
Iteration no. 0
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6543 - binary_accuracy: 0.5253
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5908 - binary_accuracy: 0.5224
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5604 - binary_accuracy: 0.5235
Iteration no. 1
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 

871/871 [==============================] - 2s 2ms/step - loss: 0.5878 - binary_accuracy: 0.5304
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5553 - binary_accuracy: 0.5304
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5345 - binary_accuracy: 0.5304
Iteration no. 15
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6319 - binary_accuracy: 0.5775
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5652 - binary_accuracy: 0.5327
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5428 - binary_accuracy: 0.5281
Iteration no. 16
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6184 - binary_accuracy: 0.5063
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5631 - binary_accuracy: 0.5327
Epoch 3/5
871/871 [=======

871/871 [==============================] - 4s 2ms/step - loss: 0.6299 - binary_accuracy: 0.5212
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5718 - binary_accuracy: 0.5626
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5250 - binary_accuracy: 0.6889
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5351 - binary_accuracy: 0.5913
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5245 - binary_accuracy: 0.5591
Iteration no. 32
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6439 - binary_accuracy: 0.5109
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5866 - binary_accuracy: 0.5235
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5594 - binary_accuracy: 0.5258
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5369 - binary_accur

871/871 [==============================] - 4s 2ms/step - loss: 0.6391 - binary_accuracy: 0.6079
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5711 - binary_accuracy: 0.5419
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5532 - binary_accuracy: 0.5224
Iteration no. 49
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6106 - binary_accuracy: 0.5144
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5638 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5317 - binary_accuracy: 0.5304
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5083 - binary_accuracy: 0.5385
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5654 - binary_accuracy: 0.6625
Iteration no. 50
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [=======

871/871 [==============================] - 2s 2ms/step - loss: 0.5241 - binary_accuracy: 0.5763
Iteration no. 64
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6335 - binary_accuracy: 0.4885
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5776 - binary_accuracy: 0.5166
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5459 - binary_accuracy: 0.5189
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5224 - binary_accuracy: 0.5339
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5142 - binary_accuracy: 0.5855
Iteration no. 65
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.5823 - binary_accuracy: 0.5264
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5465 - binary_accuracy: 0.6016
Epoch 3/5
871/871 [=======

...

871/871 [==============================] - 2s 2ms/step - loss: 0.6129 - binary_accuracy: 0.5155
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5791 - binary_accuracy: 0.5155
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5591 - binary_accuracy: 0.5155
Iteration no. 429
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6515 - binary_accuracy: 0.5344
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5760 - binary_accuracy: 0.5373
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5683 - binary_accuracy: 0.5649
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5297 - binary_accuracy: 0.5385
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5152 - binary_accuracy: 0.5396
Iteration no. 430
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [=====

871/871 [==============================] - 3s 2ms/step - loss: 0.6603 - binary_accuracy: 0.5517
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6040 - binary_accuracy: 0.5052
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5697 - binary_accuracy: 0.5052
Iteration no. 446
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6560 - binary_accuracy: 0.5482
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5958 - binary_accuracy: 0.5385
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5601 - binary_accuracy: 0.5385
Iteration no. 447
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6553 - binary_accuracy: 0.4937
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5991 - binary_accuracy: 0.5155
Epoch 3/5
871/871 [=====

871/871 [==============================] - 2s 2ms/step - loss: 0.5896 - binary_accuracy: 0.5339
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5669 - binary_accuracy: 0.5339
Iteration no. 462
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6550 - binary_accuracy: 0.5643
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5910 - binary_accuracy: 0.5304
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5518 - binary_accuracy: 0.5316
Iteration no. 463
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6583 - binary_accuracy: 0.5534
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5925 - binary_accuracy: 0.5258
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5603 - binary_accuracy: 0.5258
Iteration no. 464
7/7 [=

871/871 [==============================] - 1s 2ms/step - loss: 0.5260 - binary_accuracy: 0.5201
Epoch 5/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5165 - binary_accuracy: 0.5304
Iteration no. 479
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6399 - binary_accuracy: 0.6630
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6146 - binary_accuracy: 0.5660
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5691 - binary_accuracy: 0.5281
Iteration no. 480
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6444 - binary_accuracy: 0.4374
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5919 - binary_accuracy: 0.5408
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5526 - binary_accuracy: 0.5350
Epoch 4/5
871/871 [=====

871/871 [==============================] - 1s 2ms/step - loss: 0.5592 - binary_accuracy: 0.5235
Iteration no. 496
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6696 - binary_accuracy: 0.4529
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5992 - binary_accuracy: 0.5247
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5601 - binary_accuracy: 0.5580
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5472 - binary_accuracy: 0.5247
Epoch 5/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5288 - binary_accuracy: 0.5247
Iteration no. 497
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6841 - binary_accuracy: 0.6395
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6499 - binary_accuracy: 0.5683
Epoch 3/5
871/871 [=====

In [8]:
# Conv/LSTM architecture
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(500):
    
    model = Sequential()
    model.add(Dense(binning, activation='tanh', input_shape=(nbfeatures,binning),input_dim=nbfeatures))
    model.add(Conv1D(filters=20, kernel_size=1, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(20,input_shape=(20, nbfeatures), activation='tanh'))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6470 - binary_accuracy: 0.5149
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5578 - binary_accuracy: 0.5844
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5219 - binary_accuracy: 0.5580
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5038 - binary_accuracy: 0.5718
Iteration no. 0
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6570 - binary_accuracy: 0.4127
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5654 - binary_accuracy: 0.5224
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5377 - binary_accuracy: 0.5235
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5153 - binary_accuracy: 0.5235
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5072 - bin

871/871 [==============================] - 2s 2ms/step - loss: 0.5151 - binary_accuracy: 0.5327
Iteration no. 16
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.5993 - binary_accuracy: 0.6085
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5390 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5031 - binary_accuracy: 0.5373
Iteration no. 17
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6129 - binary_accuracy: 0.4506
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5480 - binary_accuracy: 0.5247
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5171 - binary_accuracy: 0.5327
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5011 - binary_accuracy: 0.5247
Epoch 5/5
871/871 [=======

7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6149 - binary_accuracy: 0.4839
Epoch 2/5
871/871 [==============================] - 4s 4ms/step - loss: 0.5412 - binary_accuracy: 0.5281
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.5052 - binary_accuracy: 0.5281
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4957 - binary_accuracy: 0.5270
Iteration no. 34
7/7 [==============================] - 1s 4ms/step
Epoch 1/5
871/871 [==============================] - 6s 4ms/step - loss: 0.6298 - binary_accuracy: 0.4897
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5556 - binary_accuracy: 0.5121
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.5137 - binary_accuracy: 0.5155
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4964 - binary_accuracy: 0.5959
Epoch 5/5
871/871 [==============

871/871 [==============================] - 2s 3ms/step - loss: 0.4772 - binary_accuracy: 0.5339
Epoch 5/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4856 - binary_accuracy: 0.5878
Iteration no. 50
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 5s 2ms/step - loss: 0.6630 - binary_accuracy: 0.5718
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5853 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5510 - binary_accuracy: 0.5178
Iteration no. 51
7/7 [==============================] - 1s 4ms/step
Epoch 1/5
871/871 [==============================] - 5s 2ms/step - loss: 0.6627 - binary_accuracy: 0.5775
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5771 - binary_accuracy: 0.5132
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5498 - binary_accuracy: 0.5144
Iteration no. 52
7/7 [====

871/871 [==============================] - 5s 2ms/step - loss: 0.6510 - binary_accuracy: 0.4185
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5794 - binary_accuracy: 0.5212
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5514 - binary_accuracy: 0.5201
Epoch 4/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5367 - binary_accuracy: 0.5201
Iteration no. 67
7/7 [==============================] - 1s 837us/step
Epoch 1/5
871/871 [==============================] - 5s 2ms/step - loss: 0.6134 - binary_accuracy: 0.6200
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5455 - binary_accuracy: 0.5178
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5126 - binary_accuracy: 0.5235
Iteration no. 68
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 5s 2ms/step - loss: 0.6178 - binary_accuracy: 0.4856
Epoch 2/5
871/871 [=====

...

871/871 [==============================] - 2s 2ms/step - loss: 0.5506 - binary_accuracy: 0.5316
Epoch 5/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5150 - binary_accuracy: 0.5189
Iteration no. 433
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 5s 3ms/step - loss: 0.6413 - binary_accuracy: 0.4386
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5657 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5189 - binary_accuracy: 0.5316
Epoch 4/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5070 - binary_accuracy: 0.5316
Iteration no. 434
7/7 [==============================] - 1s 3ms/step
Epoch 1/5
871/871 [==============================] - 5s 3ms/step - loss: 0.6374 - binary_accuracy: 0.5608
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5501 - binary_accuracy: 0.5212
Epoch 3/5
871/871 [=====

871/871 [==============================] - 4s 2ms/step - loss: 0.6547 - binary_accuracy: 0.5092
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5635 - binary_accuracy: 0.5247
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5479 - binary_accuracy: 0.5281
Epoch 4/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5225 - binary_accuracy: 0.5258
Epoch 5/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5163 - binary_accuracy: 0.5258
Iteration no. 451
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6272 - binary_accuracy: 0.4598
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5488 - binary_accuracy: 0.5362
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5173 - binary_accuracy: 0.5362
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5097 - binary_accu

Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6745 - binary_accuracy: 0.4742
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5917 - binary_accuracy: 0.5247
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5510 - binary_accuracy: 0.5247
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5334 - binary_accuracy: 0.5247
Iteration no. 467
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6118 - binary_accuracy: 0.5115
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5382 - binary_accuracy: 0.5327
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4982 - binary_accuracy: 0.5327
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4846 - binary_accuracy: 0.5327
Iteration no. 468
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/

871/871 [==============================] - 2s 3ms/step - loss: 0.5531 - binary_accuracy: 0.5281
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5119 - binary_accuracy: 0.5281
Epoch 4/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5070 - binary_accuracy: 0.5281
Iteration no. 483
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 5s 2ms/step - loss: 0.6539 - binary_accuracy: 0.7256
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5746 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5450 - binary_accuracy: 0.5396
Iteration no. 484
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 5s 3ms/step - loss: 0.6685 - binary_accuracy: 0.4294
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5810 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [=====

Iteration no. 499
7/7 [==============================] - 1s 2ms/step
Accuracy for ON class:  99.77777777777779 %
Accuracy for OFF class:  51.76476123311605 %
False Positive rate:  48.23523876688395 %
False Negative rate:  0.22222222222222224 %
F1 score:  0.6658871769937437


In [9]:
from sklearn.metrics import f1_score

ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(500):
    nb_filters=20
    
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)

    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="tanh",input_shape=(nbfeatures,binning)))
    model.add(Conv1D(15, 1, padding="same", activation="tanh",input_shape=(nb_filters,binning)))
    model.add(Conv1D(9, 1, padding="same", activation="tanh",input_shape=(15,binning)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(9, activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5833 - binary_accuracy: 0.5476
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5021 - binary_accuracy: 0.5752
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4924 - binary_accuracy: 0.5718
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4960 - binary_accuracy: 0.5568
Iteration no. 0
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5913 - binary_accuracy: 0.5080
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5268 - binary_accuracy: 0.5235
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5162 - binary_accuracy: 0.5235
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5085 - binary_accuracy: 0.5247
Epoch 5/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5057 - bin

871/871 [==============================] - 2s 2ms/step - loss: 0.5027 - binary_accuracy: 0.5270
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4933 - binary_accuracy: 0.5316
Iteration no. 16
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.5678 - binary_accuracy: 0.5608
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4932 - binary_accuracy: 0.5339
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4920 - binary_accuracy: 0.5442
Iteration no. 17
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6171 - binary_accuracy: 0.6515
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5348 - binary_accuracy: 0.5729
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5151 - binary_accuracy: 0.5270
Iteration no. 18
7/7 [====

871/871 [==============================] - 1s 1ms/step - loss: 0.5119 - binary_accuracy: 0.5327
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4987 - binary_accuracy: 0.5270
Iteration no. 34
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5819 - binary_accuracy: 0.5396
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5208 - binary_accuracy: 0.5178
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4991 - binary_accuracy: 0.5339
Iteration no. 35
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.6313 - binary_accuracy: 0.5982
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5191 - binary_accuracy: 0.5258
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5020 - binary_accuracy: 0.5270
Iteration no. 36
7/7 [====

871/871 [==============================] - 2s 2ms/step - loss: 0.4854 - binary_accuracy: 0.5350
Iteration no. 53
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6252 - binary_accuracy: 0.6039
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5448 - binary_accuracy: 0.5454
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5267 - binary_accuracy: 0.5293
Iteration no. 54
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6027 - binary_accuracy: 0.6137
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5337 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5170 - binary_accuracy: 0.5327
Iteration no. 55
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step

871/871 [==============================] - 2s 2ms/step - loss: 0.5831 - binary_accuracy: 0.5304
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5131 - binary_accuracy: 0.5339
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5222 - binary_accuracy: 0.5511
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5047 - binary_accuracy: 0.5339
Epoch 5/5
871/871 [==============================] - 1s 2ms/step - loss: 0.4893 - binary_accuracy: 0.5408
Iteration no. 72
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.6058 - binary_accuracy: 0.5649
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5374 - binary_accuracy: 0.5465
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5181 - binary_accuracy: 0.5626
Iteration no. 73
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [=======

...

871/871 [==============================] - 5s 3ms/step - loss: 0.6099 - binary_accuracy: 0.5011
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.5459 - binary_accuracy: 0.5327
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.5395 - binary_accuracy: 0.5580
Epoch 4/5
871/871 [==============================] - 3s 4ms/step - loss: 0.5044 - binary_accuracy: 0.6108
Epoch 5/5
871/871 [==============================] - 3s 4ms/step - loss: 0.5141 - binary_accuracy: 0.5258
Iteration no. 426
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 5s 4ms/step - loss: 0.6258 - binary_accuracy: 0.5442
Epoch 2/5
871/871 [==============================] - 3s 4ms/step - loss: 0.5450 - binary_accuracy: 0.5431
Epoch 3/5
871/871 [==============================] - 3s 4ms/step - loss: 0.5227 - binary_accuracy: 0.5339
Iteration no. 427
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [=====

871/871 [==============================] - 1s 1ms/step - loss: 0.5372 - binary_accuracy: 0.5212
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5110 - binary_accuracy: 0.5373
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5403 - binary_accuracy: 0.5373
Epoch 5/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5145 - binary_accuracy: 0.5212
Iteration no. 442
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5864 - binary_accuracy: 0.5568
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5072 - binary_accuracy: 0.6361
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5028 - binary_accuracy: 0.5258
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4969 - binary_accuracy: 0.5270
Iteration no. 443
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [=====

871/871 [==============================] - 1s 1ms/step - loss: 0.5088 - binary_accuracy: 0.5855
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4938 - binary_accuracy: 0.5488
Iteration no. 459
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5954 - binary_accuracy: 0.6958
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5177 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5010 - binary_accuracy: 0.5247
Iteration no. 460
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5876 - binary_accuracy: 0.6022
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5122 - binary_accuracy: 0.5385
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4936 - binary_accuracy: 0.5316
Iteration no. 461
7/7 [=

871/871 [==============================] - 1s 1ms/step - loss: 0.5021 - binary_accuracy: 0.6165
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4608 - binary_accuracy: 0.6452
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4642 - binary_accuracy: 0.5993
Epoch 5/5
871/871 [==============================] - 1s 1ms/step - loss: 0.4602 - binary_accuracy: 0.5695
Iteration no. 477
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.6124 - binary_accuracy: 0.4897
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5374 - binary_accuracy: 0.5293
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5456 - binary_accuracy: 0.5362
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5205 - binary_accuracy: 0.5293
Epoch 5/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5115 - binary_accu

871/871 [==============================] - 1s 1ms/step - loss: 0.4919 - binary_accuracy: 0.5247
Iteration no. 495
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.5957 - binary_accuracy: 0.5218
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5226 - binary_accuracy: 0.5224
Epoch 3/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5092 - binary_accuracy: 0.5327
Epoch 4/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5024 - binary_accuracy: 0.5224
Epoch 5/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5035 - binary_accuracy: 0.5235
Iteration no. 496
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 1ms/step - loss: 0.6255 - binary_accuracy: 0.5098
Epoch 2/5
871/871 [==============================] - 1s 1ms/step - loss: 0.5334 - binary_accuracy: 0.5258
Epoch 3/5
871/871 [=====